In [177]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sympy import *
from sympy.core.numbers import NaN
from copy import deepcopy
from IPython.display import display, HTML
from sympy.core.numbers import NaN


def log_barriers(func: str, restrictions: list, start_point: tuple = tuple(), accuracy:float = 10**(-6), max_steps: int=500):
    tao = 1
    v = 10
    dataset = []
    for i in range(len(restrictions)):
        restrictions[i] = restrictions[i][:restrictions[i].index('>')].replace(' ', '')

    phi = f'{tao}*({func})'
    for exp in restrictions:
        phi += f' - log({exp})'
        
    X = Matrix([sympify(phi)])
    symbs = list(sympify(phi).free_symbols)

    if len(start_point) == 0:
        start_point = first_phase(restrictions, symbs)
    if start_point == 'Введенные ограничения не могут использоваться вместе':
        return start_point
    elif start_point == 'Невозможно подобрать внутреннюю точку для данного метода':
        return start_point

    try:
        res = sympify(func).subs(list(zip(symbs, start_point)))
        dataset.append([*start_point, res])
    except:
        return 'Введена первоначальная точка, которая не удовлетворяет неравенствам'
    Y = Matrix(list(symbs))
    
    df = X.jacobian(Y).T
    ddf = df.jacobian(Y)
    
    lst_for_subs =  list(zip(symbs, start_point))
    dfx0 = df.subs(lst_for_subs)
    ddfx0 = ddf.subs(lst_for_subs)
    
    xk = ddfx0.inv() @ dfx0
    next_point = [start_point[i]-xk[i] for i in range(len(start_point))]
    tao = tao*v
    
    
    res_new = sympify(func).subs(list(zip(symbs, next_point)))
    dataset.append([*next_point, res_new])
    if type(res_new) == NaN:
        return start_point, dataset
        
    steps = 1
    while abs(res_new - res) > accuracy and max_steps > steps:
        phi = f'{tao}*({func})'
        for exp in restrictions:
            phi += f' - log({exp})'
        
        X = Matrix([sympify(phi)])
        symbs = list(sympify(phi).free_symbols)
        Y = Matrix(list(symbs))

        df = X.jacobian(Y).T
        ddf = df.jacobian(Y)

        lst_for_subs =  list(zip(symbs, start_point))
        dfx0 = df.subs(lst_for_subs)
        ddfx0 = ddf.subs(lst_for_subs)

        xk = ddfx0.inv() @ dfx0
        old_point = deepcopy(next_point)
        next_point = [next_point[i]-xk[i] for i in range(len(next_point))]
        res = deepcopy(res_new)
        res_new = sympify(func).subs(list(zip(symbs, next_point)))
        dataset.append([*next_point, res_new])
        if type(res_new) == NaN:
            return old_point, dataset

        tao = tao*v
        steps += 1
        

    return next_point, dataset



In [178]:
log_barriers('x**2 + y**2', ['-2*x + y + 2 >= 0'], (0, 0))

([16167045269545329836542/89748295655001222469685,
  -32334090539090659673084/89748295655001222469685],
 [[0, 0, 0],
  [2/13, -4/13, 20/169],
  [196/1105, -392/1105, 38416/244205],
  [31998/177905, -63996/177905, 1023872004/6330037805],
  [10259992/56965181, -20519984/56965181, 526337179200320/3245031846362761],
  [820964590322/4557499305905,
   -1641929180644/4557499305905,
   673982858562569296063684/4154159984664911353573805],
  [131356978415832684/729204446444105905,
   -262713956831665368/729204446444105905,
   17254655778537533620654725082643856/106347824942770983407787462971173805],
  [16167045269545329836542/89748295655001222469685,
   -32334090539090659673084/89748295655001222469685,
   261373352747528026669119153470218924438517764/1610951314595502261633287462186356426148799845]])

In [179]:
dataset_rec = log_barriers('x**2 + y**2', ['-2*x + y + 2 >= 0'], (0, 0))[1]

In [180]:
dataset_rec

[[0, 0, 0],
 [2/13, -4/13, 20/169],
 [196/1105, -392/1105, 38416/244205],
 [31998/177905, -63996/177905, 1023872004/6330037805],
 [10259992/56965181, -20519984/56965181, 526337179200320/3245031846362761],
 [820964590322/4557499305905,
  -1641929180644/4557499305905,
  673982858562569296063684/4154159984664911353573805],
 [131356978415832684/729204446444105905,
  -262713956831665368/729204446444105905,
  17254655778537533620654725082643856/106347824942770983407787462971173805],
 [16167045269545329836542/89748295655001222469685,
  -32334090539090659673084/89748295655001222469685,
  261373352747528026669119153470218924438517764/1610951314595502261633287462186356426148799845]]

In [181]:
dataset_rec = pd.DataFrame(dataset_rec)
dataset_rec.rename(columns={0: 'x_1', 1: 'x_2', 2: 'f'}, inplace=True)

In [182]:
dataset_rec.head()

,x_1,x_2,f
0,0,0,0
1,2/13,-4/13,20/169
2,196/1105,-392/1105,38416/244205
3,31998/177905,-63996/177905,1023872004/6330037805
4,10259992/56965181,-20519984/56965181,526337179200320/3245031846362761


In [183]:
dataset_rec['x_1'] = dataset_rec['x_1'].apply(float)
dataset_rec['x_2'] = dataset_rec['x_2'].apply(float)
dataset_rec['f'] = dataset_rec['f'].apply(float)

In [184]:
dataset_rec.head()

,x_1,x_2,f
0,0.000000,0.000000,0.000000
1,0.153846,-0.307692,0.118343
2,0.177376,-0.354751,0.157310
3,0.179860,-0.359720,0.161748
4,0.180110,-0.360220,0.162198


In [185]:
x = [i for i in dataset_rec['x_1'].tolist()]
y = [i for i in dataset_rec['x_2'].tolist()]

In [186]:
x_min, x_max = min(x), max(x)
y_min, y_max = min(y), max(y)

In [187]:
X = np.linspace(x_min, x_max, 100)
Y = np.linspace(y_min, y_max, 100)

In [188]:
X, Y = np.meshgrid(X, Y)

In [189]:
f = lambda x,y: x**2 + y**2

In [190]:
Z = f(X, Y)

In [191]:
scatter = go.Scatter3d(x=x, y=y, z=dataset_rec['f'], marker={'size':3})

surface = go.Surface(z=Z, x=X, y=Y, colorscale='RdBu', opacity=0.3,  showscale=True)

In [192]:
fig = go.Figure(data=[scatter, surface])
fig.show()